In [35]:
import math
import hashlib
import inspect
import json
import pandas as pd
import numpy as np
import re
from hashlib import sha256
from collections import Counter
import string
import os
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
from spacy.matcher import Matcher
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('data/covid19_data.csv')
df_test_pred = pd.read_csv('data/covid19_unlabelled_test.csv')

In [3]:
df_train.shape #(5531, 15)
df_train.head()

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Delfi Melo Detektorius (Lie Detector),Lithuania,FALSE,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,poynter,Baltarusija atšventė nesisaugodama nuo koronav...,Gegužės 9-ąją Baltarusijos sostinėje Minske su...,NaN,lt
1,AfricaCheck,"United States, South Africa",False,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,poynter,"No, Gaddafi didn’t predict coronavirus, but di...",As the world struggles to bring the new corona...,NaN,en
2,AFP,Hong Kong,False,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,poynter,"This video shows tower blocks in Shanghai, not...","It has been viewed more than 200,000 times., T...",NaN,en
3,FactCrescendo,India,False,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,poynter,"This blast took place in Tiajin, China in 2015...","The fact behind every news!, Ever since, the W...",NaN,en
4,Newtral.es,"Spain, Peru, United States, Mexico",False,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,poynter,El dióxido de cloro no cura el coronavirus; al...,Circula un video de Youtube titulado: “El Coro...,NaN,es


In [4]:
class_value = df_train['class'].value_counts(dropna=False)

In [5]:
class_value.index
class_value.head(20).sum() #5416 - 98%

5416

In [15]:
df_train['label'] = df_train['class'].replace({'False' : 'false',
'FALSE' : 'false',
'Misleading' : 'false',
'MISLEADING' : 'false',
'Mostly false' : 'false',
'Partly false': 'false',
'misleading' : 'false',
'No evidence' : 'true',
'Mostly False' : 'false',
'Mixture': 'false',
'True': 'true',
'Explanatory': 'true',
'No Evidence': 'true',
'News': 'true',
'PARTLY FALSE': 'false',
'Unproven': 'false',
'MOSTLY FALSE': 'false',
'Partly False': 'false',
'Miscaptioned': 'false',
'partly false': 'false',
'mostly false' : 'false',
'Mostly True': 'true',
'MOSTLY TRUE': 'true',
'Misattributed': 'false',
"(Org. doesn't apply rating)": 'false',
'HALF TRUE': 'true',
'Correct Attribution' :'true',
'Partially false': 'false',
'Labeled Satire': 'false',
'Fake' : 'false',
'NO EVIDENCE': 'true',
'false' : 'false',
'Two Pinocchios': 'false',
'Scam' : 'false',
'no evidence': 'true',
'Half True': 'true',
'PARTLY TRUE': 'true',
'half true': 'true',
'Correct': 'true',
'mislEADING' : 'false',
'Suspicions': 'false',
'Not true' : 'false',
'nan': 'false',
'Partly FALSE': 'false',
'Misleading/False' : 'false',
'PANTS ON FIRE': 'false',
'Partially true': 'true',
'Mixed': 'false',
'IN DISPUTE': 'false',
'Unverified': 'false',
'HALF TRUTH': 'true',
'Collections': 'false',
'Partially correct': 'true',
'MiSLEADING' : 'false',
'Mostly true': 'true',
'True but': 'true',
'false and misleading' : 'false',
'Partly true': 'true',
'Misinformation / Conspiracy theory' : 'false',
'Unlikely' : 'false',
'Fake news' : 'false'})

In [16]:
df_train['label'].value_counts(dropna=False)

false    5329
true      200
NaN         2
Name: label, dtype: int64

In [41]:
df_train['label'] = df_train['label'].fillna('false')
df_train['label'].value_counts(dropna=False)

false    5331
true      200
Name: label, dtype: int64

In [42]:
X = pd.DataFrame(df_train['title'])
y = df_train['label']

X.shape

(5531, 1)

In [33]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# X_test.head()

In [43]:
# Preprocess text
# remove: stopwords and punctuation

filter_stopwords = True
remove_punkt = True

en_stopwords = stopwords.words('english')
tokenizer =  WordPunctTokenizer()

def preprocess(text, stopwords=False, remove_punkt=False, truncate=None):
    tokens = tokenizer.tokenize(text)
    tokens = [tok.lower() for tok in tokens]
    if stopwords:
        tokens = [tok for tok in tokens if tok not in en_stopwords]
    if remove_punkt:
        tokens = [tok for tok in tokens if tok not in string.punctuation]
    if truncate:
        tokens = tokens[:truncate]
    #tokens = [PorterStemmer().stem(tok) for tok in tokens]
    text_preprocessed = ' '.join(tokens)
    return text_preprocessed

In [49]:
X['title_preprocessed'] = X['title'].apply(preprocess, args=(filter_stopwords, remove_punkt, None))

In [50]:
X_train_preproc = pd.DataFrame(X['title_preprocessed'])

In [53]:
X_train_preproc.head()

,title_preprocessed
0,claims coronavirus fake belarus coping well “ ...
1,muammar gaddafi predicted current coronavirus ...
2,video shows quarantined people building shanghai
3,video shows massive explosion wuhan china cont...
4,chlorine dioxide cures coronavirus
